# Setup Environment

In [1]:
!pip install -q torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 39.0 MB/s eta 0:00:00


In [2]:
import os
import json
import torch
import torch.nn.functional as F
from torch.nn import Module, Embedding, Linear
from torch_geometric.data import Data, Dataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import RGCNConv, global_mean_pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
import random
import time
import numpy as np

In [3]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Dataset

In [4]:
class PrecomputedGraphDataset(Dataset):
    def __init__(self, root_dir, graph_file, vocab_file, transform=None, pre_transform=None):
        self.root_dir = root_dir
        
        # Tải dữ liệu đồ thị và từ điển từ các file JSON
        with open(graph_file, 'r') as f:
            self.graph_data_dict = json.load(f)
        with open(vocab_file, 'r') as f:
            self.vocabdict = json.load(f)
            
        self.all_paths = list(self.graph_data_dict.keys())
        
        # Xác định nhãn dựa trên đường dẫn file
        self.labels = []
        valid_paths = []
        for path in self.all_paths:
            # Chuẩn hóa đường dẫn để hoạt động trên mọi HĐH
            normalized_path = path.replace('\\', '/')
            if '/smelly/' in normalized_path:
                self.labels.append(1)
                valid_paths.append(path)
            elif '/non-smelly/' in normalized_path:
                self.labels.append(0)
                valid_paths.append(path)
        
        # Chỉ giữ lại các đường dẫn có nhãn hợp lệ
        self.all_paths = valid_paths

        super(PrecomputedGraphDataset, self).__init__(root_dir, transform, pre_transform)
        
    def len(self):
        return len(self.all_paths)

    def get(self, idx):
        # Lấy đường dẫn và nhãn tương ứng với chỉ số
        path = self.all_paths[idx]
        label = self.labels[idx]
        
        # Lấy dữ liệu đồ thị đã được tính toán trước từ dictionary
        graph_info = self.graph_data_dict[path]
        
        # Giải nén dữ liệu
        graph_tensors, astlength = graph_info
        x_list, edge_index_list, edge_attr_list = graph_tensors
        
        # Chuyển đổi list thành tensor của PyTorch
        x = torch.tensor(x_list, dtype=torch.long)
        edge_index = torch.tensor(edge_index_list, dtype=torch.long)
        edge_attr = torch.tensor(edge_attr_list, dtype=torch.long)
        
        # Kiểm tra và sửa lỗi shape nếu cần
        if edge_index.dim() > 1 and edge_index.size(0) != 2 and edge_index.size(1) == 2:
            edge_index = edge_index.t() # Chuyển vị nếu shape là [num_edges, 2]

        # Tạo đối tượng Data của PyTorch Geometric
        data = Data(
            x=x,
            edge_index=edge_index,
            edge_type=edge_attr.squeeze(-1), # RGCNConv yêu cầu edge_type là 1D tensor
            y=torch.tensor([label], dtype=torch.float)
        )
        return data

# Model

In [5]:
class RGCNCodeSmellClassifier(Module):
    def __init__(self, vocab_size, num_relations, embedding_dim=256, gcn_hidden_dim=256, linear_hidden_dim=128):
        super(RGCNCodeSmellClassifier, self).__init__()
        self.embedding = Embedding(vocab_size, embedding_dim)
        self.conv1 = RGCNConv(embedding_dim, gcn_hidden_dim, num_relations)
        self.conv2 = RGCNConv(gcn_hidden_dim, gcn_hidden_dim, num_relations)
        self.conv3 = RGCNConv(gcn_hidden_dim, gcn_hidden_dim, num_relations)
        self.conv4 = RGCNConv(gcn_hidden_dim, gcn_hidden_dim, num_relations)
        self.conv5 = RGCNConv(gcn_hidden_dim, gcn_hidden_dim, num_relations)
        self.fc1 = Linear(gcn_hidden_dim, linear_hidden_dim)
        self.fc2 = Linear(linear_hidden_dim, 1)

    def forward(self, data):
        x, edge_index, edge_type, batch = data.x, data.edge_index, data.edge_type, data.batch
        x = self.embedding(x.squeeze(-1))
        x = F.relu(self.conv1(x, edge_index, edge_type))
        x = F.relu(self.conv2(x, edge_index, edge_type))
        x = F.relu(self.conv3(x, edge_index, edge_type))
        x = F.relu(self.conv4(x, edge_index, edge_type))
        x = F.relu(self.conv5(x, edge_index, edge_type))
        graph_embedding = global_mean_pool(x, batch)
        x = F.relu(self.fc1(graph_embedding))
        x = F.dropout(x, p=0.5, training=self.training)
        out = self.fc2(x)
        return out

# Train & Evaluation

In [6]:
def train(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    dataset_size = len(loader.dataset)
    for data in tqdm(loader, desc="Training"):
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = criterion(out, data.y.view_as(out))
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
    return total_loss / dataset_size

def evaluate(model, loader, device):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for data in tqdm(loader, desc="Evaluating"):
            data = data.to(device)
            out = model(data)
            preds = (torch.sigmoid(out) > 0.5).cpu().numpy()
            labels = data.y.cpu().numpy()
            all_preds.extend(preds.flatten())
            all_labels.extend(labels.flatten())

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, zero_division=0)
    recall = recall_score(all_labels, all_preds, zero_division=0)
    f1 = f1_score(all_labels, all_preds, zero_division=0)
    return accuracy, precision, recall, f1

# Train Loop

In [7]:
def main():
    # --- Cấu hình ---
    DATA_ROOT = "/kaggle/input/sdwllm-code-smell-long-method"
    GRAPH_DATA_FILE = os.path.join(DATA_ROOT, "graph.json")
    VOCAB_FILE = os.path.join(DATA_ROOT, "vocabdict.json")
    
    NUM_EPOCHS = 10
    BATCH_SIZE = 32
    # godclass 0.0005
    # dataclass 0.0005
    # featureenvy 0.0001
    # longmethod 0.0005
    LEARNING_RATE = 0.0005

    # --- Tải từ điển và Dataset ---
    print("Tải dữ liệu từ file JSON...")
    dataset = PrecomputedGraphDataset(root_dir=DATA_ROOT, graph_file=GRAPH_DATA_FILE, vocab_file=VOCAB_FILE)
    
    # --- Chia dữ liệu thành 3 tập: 70% train, 20% validation, 10% test ---
    print("Chia dữ liệu thành các tập train, validation, và test...")
    indices = list(range(len(dataset)))
    labels = np.array(dataset.labels)
    
    # Bước 1: Chia 70% train và 30% còn lại (dành cho validation và test)
    train_indices, temp_indices, train_labels, temp_labels = train_test_split(
        indices, labels, test_size=0.3, random_state=42, stratify=labels
    )
    # Bước 2: Chia 30% còn lại thành 20% validation và 10% test
    # Tỷ lệ test trong tập temp là 10/30 = 1/3
    valid_indices, test_indices, valid_labels, test_labels = train_test_split(
        temp_indices, temp_labels, test_size=(1/3), random_state=42, stratify=temp_labels
    )

    print(f"Tổng số mẫu: {len(dataset)}")
    print(f"Số mẫu train: {len(train_indices)} (~{len(train_indices)/len(dataset)*100:.1f}%)"
          f" -> Smelly(1): {np.sum(train_labels == 1)}, Non-smelly(0): {np.sum(train_labels == 0)}")
    print(f"Số mẫu validation: {len(valid_indices)} (~{len(valid_indices)/len(dataset)*100:.1f}%)"
          f" -> Smelly(1): {np.sum(valid_labels == 1)}, Non-smelly(0): {np.sum(valid_labels == 0)}")
    print(f"Số mẫu test: {len(test_indices)} (~{len(test_indices)/len(dataset)*100:.1f}%)"
          f" -> Smelly(1): {np.sum(test_labels == 1)}, Non-smelly(0): {np.sum(test_labels == 0)}")

    train_dataset = torch.utils.data.Subset(dataset, train_indices)
    valid_dataset = torch.utils.data.Subset(dataset, valid_indices)
    test_dataset = torch.utils.data.Subset(dataset, test_indices)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    # --- Khởi tạo mô hình, optimizer và loss ---
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Sử dụng thiết bị: {device}")
    
    vocab_size = len(dataset.vocabdict)
    # 9 loại cạnh, có forward/backward nên x2:
    # Child/Parent, NextSib/PrevSib, NextToken/PrevToken, NextUse/PrevUse,
    # CondTrue/Rev, CondFalse/Rev, WhileExec/WhileNext, ForExec/ForNext, NextStmt/PrevStmt
    num_relations = 18

    model = RGCNCodeSmellClassifier(vocab_size=vocab_size, num_relations=num_relations).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    
    # Tính toán trọng số cho lớp thiểu số (positive class - smelly)
    num_negatives = np.sum(train_labels == 0)
    num_positives = np.sum(train_labels == 1)
    
    if num_positives > 0:
        # godclass /2
        # dataclass /2
        # featureenvy *1
        # longmethod /2
        pos_weight_value = num_negatives / num_positives / 2
        pos_weight = torch.tensor([pos_weight_value], dtype=torch.float).to(device)
        print(f"Áp dụng trọng số cho lớp Smelly (positive): {pos_weight.item():.2f}")
    else:
        pos_weight = None # Không có mẫu positive trong tập train
        print("Không tìm thấy mẫu Smelly trong tập train. Không áp dụng trọng số.")

    criterion = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight)

    # --- Vòng lặp huấn luyện ---
    print("Bắt đầu huấn luyện...")
    start_time = time.time()
    for epoch in range(1, NUM_EPOCHS + 1):
        train_loss = train(model, train_loader, optimizer, criterion, device)
        
        # Đánh giá trên tập validation sau mỗi epoch
        val_accuracy, val_precision, val_recall, val_f1 = evaluate(model, valid_loader, device)
        
        print(f'Epoch: {epoch:02d}, Loss: {train_loss:.4f}, '
              f'Val Acc: {val_accuracy:.4f}, Val Precision: {val_precision:.4f}, '
              f'Val Recall: {val_recall:.4f}, Val F1: {val_f1:.4f}')
    training_time = time.time() - start_time
    mins, secs = divmod(training_time, 60)
    
    # --- Đánh giá cuối cùng trên tập test ---
    print("\nĐã huấn luyện xong. Đánh giá trên tập test cuối cùng...")
    test_accuracy, test_precision, test_recall, test_f1 = evaluate(model, test_loader, device)
    
    print(f'========== Final Test Results ==========')
    print(f'Test Accuracy:  {test_accuracy:.4f}')
    print(f'Test Precision: {test_precision:.4f}')
    print(f'Test Recall:    {test_recall:.4f}')
    print(f'Test F1-Score:  {test_f1:.4f}')
    print(f'Training Time:  {round(mins)}m {round(secs)}s')
    print(f'======================================')

In [8]:
main()

Tải dữ liệu từ file JSON...
Chia dữ liệu thành các tập train, validation, và test...
Tổng số mẫu: 2234
Số mẫu train: 1563 (~70.0%) -> Smelly(1): 169, Non-smelly(0): 1394
Số mẫu validation: 447 (~20.0%) -> Smelly(1): 49, Non-smelly(0): 398
Số mẫu test: 224 (~10.0%) -> Smelly(1): 24, Non-smelly(0): 200
Sử dụng thiết bị: cuda
Áp dụng trọng số cho lớp Smelly (positive): 4.12
Bắt đầu huấn luyện...


Evaluating: 100%|██████████| 14/14 [00:00<00:00, 20.74it/s]


Epoch: 01, Loss: 0.6142, Val Acc: 0.8814, Val Precision: 0.4773, Val Recall: 0.8571, Val F1: 0.6131


Evaluating: 100%|██████████| 14/14 [00:00<00:00, 22.88it/s]


Epoch: 02, Loss: 0.3320, Val Acc: 0.8859, Val Precision: 0.4889, Val Recall: 0.8980, Val F1: 0.6331


...

Epoch: 09, Loss: 0.0583, Val Acc: 0.9038, Val Precision: 0.5417, Val Recall: 0.7959, Val F1: 0.6446


Evaluating: 100%|██████████| 14/14 [00:00<00:00, 23.02it/s]


Epoch: 10, Loss: 0.0493, Val Acc: 0.9105, Val Precision: 0.5652, Val Recall: 0.7959, Val F1: 0.6610

Đã huấn luyện xong. Đánh giá trên tập test cuối cùng...


Evaluating: 100%|██████████| 7/7 [00:00<00:00, 23.19it/s]


========== Final Test Results ==========
Test Accuracy:  0.9554
Test Precision: 0.7188
Test Recall:    0.9583
Test F1-Score:  0.8214
Training Time:  0m 43s
